# Artificial Visual Imagination 
## Text to Image with BigGAN + CLIP + CMA-ES

---

BIGCLIP [j.mp/bigclip](https://j.mp/bigclip) by Eyal Gruss [@eyaler](https://twitter.com/eyaler) [eyalgruss.com](https://eyalgruss.com)


Modified to run on nautilus.optiputer.net by robert.twomey@gmail.com

# 1. Generating from Saved Vector

from https://github.com/huggingface/pytorch-pretrained-BigGAN#usage

In [3]:
from IPython.display import HTML, clear_output
from PIL import Image
from IPython.display import Image as JupImage
import numpy as np
import nltk
nltk.download('wordnet')
from scipy.stats import truncnorm

# from biggan
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.WARNING)

# Load pre-trained model tokenizer (vocabulary)
model = BigGAN.from_pretrained('biggan-deep-512')
print("loaded bigGAN")

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


loaded bigGAN


In [ ]:
# def save(out, name):
#     with torch.no_grad():
#         out = out.cpu().numpy()
#     img = convert_to_images(out)[0]
#     imageio.imwrite(name, np.asarray(img))

In [ ]:
# from IPython.display import Image
# display(Image("output_0.png"))

Generate from a set of vectors

In [ ]:
# index = 6

# # random seeding
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)


# for index in range(50):
    
#     # Prepare a input
#     truncation = 1.0
#     noise_vectors = np.loadtxt('/home/jovyan/work/output_20210222_194102/class_00008.txt')
#     class_vectors = np.loadtxt('/home/jovyan/work/output_20210222_194102/class_00009.txt')

#     noise_vector = noise_vectors[index]
#     class_vector = class_vectors[index]

#     # expand dims
#     noise_vector = np.expand_dims(noise_vector, axis=0)
#     class_vector = np.expand_dims(class_vector, axis=0)

#     # All in tensors
#     noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
#     class_vector = torch.tensor(class_vector, dtype=torch.float32)

#     print(noise_vector.shape, noise_vector.dtype, class_vector.shape, class_vector.dtype)

#     # If you have a GPU, put everything on cuda
#     noise_vector = noise_vector.to('cuda')
#     noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
#     class_vector = class_vector.to('cuda')
#     class_vector = class_vector.softmax(dim=-1)
#     model.to('cuda')

#     # Generate an image
#     with torch.no_grad():
#         output = model(noise_vector, class_vector, truncation)

#     # If you have a GPU put back on CPU
#     output = output.to('cpu')

#     # # If you have a sixtel compatible terminal you can display the images in the terminal
#     # # (see https://github.com/saitoha/libsixel for details)
#     # # display_in_terminal(output)
#     save_as_images(output, "output_%s" % index)

#     # # display(Image())

#     # # Save results as png images
#     # # save_as_images(output)

In [ ]:
# for i in range(50):
#     print("Image %s:" %i)
#     display(Image('output_%s_0.png' % i))

In [ ]:
# # from huggingface
# # # Prepare a input
# # truncation = 0.4
# class_vector = one_hot_from_names(['soap bubble', 'coffee', 'mushroom'], batch_size=3)
# noise_vector = truncated_noise_sample(truncation=truncation, batch_size=3)
# print(class_vector.shape, class_vector.dtype)
# print(noise_vector.shape, class_vector.dtype)



my stuff

In [ ]:
# noise_vec = np.loadtxt('/home/jovyan/work/output_20210221_175545/noise_00009.txt')
# class_vec = np.loadtxt('/home/jovyan/work/output_20210221_175545/class_00009.txt')
# # print(noise_vec, class_vec)

# noise_vec = torch.tensor(noise_vec, dtype=torch.float32).cuda()
# class_vec = torch.tensor(class_vec, dtype=torch.float32).cuda()

# # vectors = torch.cat([noise_vector,class_vector], dim=1)
# # print(vectors)
# out, class_vector_norm = get_output(noise_vec, class_vec)

# 2. Latent+Class Interpolation

In [4]:
from numpy import asarray
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from numpy import arccos
from numpy import clip
from numpy import dot
from numpy import sin
from numpy import linspace
from numpy.linalg import norm
import os
import glob

# from
# https://discuss.pytorch.org/t/help-regarding-slerp-function-for-generative-model-sampling/32475/4

# spherical linear interpolation (slerp)
def slerp(val, low, high):
    omega = arccos(clip(dot(low/norm(low), high/norm(high)), -1, 1))
    so = sin(omega)
    if so == 0:
        # L'Hopital's rule/LERP
        return (1.0-val) * low + val * high
    return sin((1.0-val)*omega) / so * low + sin(val*omega) / so * high
 
# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=10):
    # interpolate ratios between the points
    ratios = np.linspace(0, 1, num=n_steps)
    # linear interpolate vectors
    vectors = list()
    for ratio in ratios:
        v = slerp(ratio, p1, p2)
        vectors.append(v)
    return np.asarray(vectors)

Interpolate between a set of vectors:

In [30]:
np.random.RandomState(1)
np.random.seed(1)
torch.manual_seed(1)

# Prepare a input
truncation = 1.0
resultsbase = '/home/jovyan/work/results/'
workbase = '/home/jovyan/work/'
interpbase = '/home/jovyan/work/interpolation'

# sculptures: 

# class_filenames = [
# "work/visual-imagination/CLIP/class_sculpture television buddha.txt",
# "work/visual-imagination/CLIP/class_a buddha sculpture with television in the grass.txt",
# "work/visual-imagination/CLIP/class_television buddha sculpture with grass.txt",
# "work/visual-imagination/CLIP/class_old television in grass with buddha sculpture by pacific ocean.txt",
# "work/visual-imagination/CLIP/class_old television in long grass with buddha sculpture by pacific ocean.txt"
# ]

# noise_filenames = [
# "work/visual-imagination/CLIP/noise_sculpture television buddha.txt",
# "work/visual-imagination/CLIP/noise_a buddha sculpture with television in the grass.txt",
# "work/visual-imagination/CLIP/noise_television buddha sculpture with grass.txt",
# "work/visual-imagination/CLIP/noise_old television in grass with buddha sculpture by pacific ocean.txt",
# "work/visual-imagination/CLIP/noise_old television in long grass with buddha sculpture by pacific ocean.txt"
# ]

prompts = [
    'a photo of wild tarragon',
    'a drawing of wild tarragon, a tasteless plant',
    'a painting of farm hands, a kind of laborer',
    'a painting of a farmer’s hands',
    'a self-portrait of Artemisia Gentileschi, artist',
    'artemisia Gentileschi is a dragon',
    'a painting of Artemisia Gentileschi as a dragon',
    'a photo of the dragon Artemisia Gentileschi',
    'a portrait of artist as dragon',
    'a drawing of a dragon',
    'a painting of uprooted rhizome as a dragon',
    'a sketch of a rhizome, uprooted',
    'an image of a plant rising',
    'a drawing of plant roots and mycorrhizal fungi',
    'an image of growing wiser',
    'a painting of wise plants',
    'a drawing of plant wisdom',
    'a photo of a plant hiding',
    'a drawing of hiding from elders',
    'a painting of Susanna and the Elders',
    'an image of creeps',
    'a painting of gazing creeps',
    'a painting of groping creeps',
    'a painting of invasive elders',
    'a photo of perverse hope',
    'a painting of your hatred',
    'a drawing of killing a mosquito',
    'a painting of a mosquito, a kind of corpse',
    'a drawing of malaria',
    'a sketch of salted fields',
    'a photo of dancers',
    'a painting of dancers in a field',
    'an image of your spit',
    'a photo of standing too close',
    'a painting of someone standing too close',
    'a drawing of an oak sapling',
    'a painting of an oak in an empty field',
    'a photo of growing',
    'an image of growing wilder',
    'a painting of growing stronger',
    'a photo of a hand holding high',
    'a painting of a hand holding the head of Holofernes',
    'a painting of the head of Holofernes',
    'a drawing of a head, blood-rooted',
    'an image of a bloody root',
    'a painting of autumn gold',
    'a photo of a golden gown',
    'an image of a mouth tasting',
    'a sketch of a mouth',
    'a drawing of taste',
    'a painting of the taste of nothing',
    'a photograph of being invisible',
    'a drawing of your renown',
    'a painting of a renowned artist',
    'a portrait of the artist',
    'a self-portrait of Artemisia Gentileschi as tarragon'
]

safe_prompts = [prompt.replace(' ', '_') for prompt in prompts]

# print(safe_prompts)

def get_class_file(path, prompt):
    result = glob.glob(path+'%s_*_class.txt'%prompt)
    return(result)

def get_noise_file(path, prompt):
    result = glob.glob(path+'%s_*_noise.txt'%prompt)
    return(result)

# print(get_class_file(workbase, safe_prompts[0]))
# print(get_noise_file(workbase, safe_prompts[0]))

In [41]:
class_filenames = [get_class_file(resultsbase, prompt)[0] for prompt in safe_prompts]
noise_filenames = [get_noise_file(resultsbase, prompt)[0] for prompt in safe_prompts]

# print(class_filenames, noise_filenames)

['/home/jovyan/work/results/a_photo_of_wild_tarragon_20210227_140023_class.txt', '/home/jovyan/work/results/a_drawing_of_wild_tarragon,_a_tasteless_plant_20210227_140504_class.txt', '/home/jovyan/work/results/a_painting_of_farm_hands,_a_kind_of_laborer_20210227_140945_class.txt', '/home/jovyan/work/results/a_painting_of_a_farmer’s_hands_20210227_141432_class.txt', '/home/jovyan/work/results/a_self-portrait_of_Artemisia_Gentileschi,_artist_20210227_141920_class.txt', '/home/jovyan/work/results/artemisia_Gentileschi_is_a_dragon_20210227_142403_class.txt', '/home/jovyan/work/results/a_painting_of_Artemisia_Gentileschi_as_a_dragon_20210227_142843_class.txt', '/home/jovyan/work/results/a_photo_of_the_dragon_Artemisia_Gentileschi_20210227_143328_class.txt', '/home/jovyan/work/results/a_portrait_of_artist_as_dragon_20210227_143807_class.txt', '/home/jovyan/work/results/a_drawing_of_a_dragon_20210227_151124_class.txt', '/home/jovyan/work/results/a_painting_of_uprooted_rhizome_as_a_dragon_20210

In [42]:
num_steps = 200 #300
len_hold = 30

class_inputs = [np.loadtxt(filename) for filename in class_filenames]
noise_inputs = [np.loadtxt(filename) for filename in noise_filenames]

# print(class_inputs, noise_inputs)

count = 0

for i in range(len(class_inputs)):
    
    # interpolate
    noises = interpolate_points(noise_inputs[i], noise_inputs[(i+1)%len(class_inputs)], num_steps)
    classes = interpolate_points(class_inputs[i], class_inputs[(i+1)%len(class_inputs)], num_steps)

    for j in range(num_steps):

        # expand dims
        noise_vector = np.expand_dims(noises[j], axis=0)
        class_vector = np.expand_dims(classes[j], axis=0)

        # convert to tensors
        noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
        class_vector = torch.tensor(class_vector, dtype=torch.float32)
    
        # If you have a GPU, put everything on cuda
        noise_vector = noise_vector.to('cuda')
        noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
        class_vector = class_vector.to('cuda')
        class_vector = class_vector.softmax(dim=-1)
        model.to('cuda')

        # Generate an image
        with torch.no_grad():
            output = model(noise_vector, class_vector, truncation)

        # If you have a GPU put back on CPU
        output = output.to('cpu')

        # # If you have a sixtel compatible terminal you can display the images in the terminal
        # # (see https://github.com/saitoha/libsixel for details)
    
        # "hold" on the first frame for fixed time
        if j == 0:
            for k in range(len_hold-1):
                save_as_images(output, interpbase+"/output_%05d" % count)
                count = count + 1
    #     clear_output()
    #     display(JupImage("output_%05d_0.png" % i))
    
        save_as_images(output, interpbase+"/output_%05d" % count)
        count = count + 1

# 2. Create Video

In [ ]:
# cd $outpath

In [43]:
# pwd

'/home/jovyan/work/visual-imagination'

In [45]:
# print(count)

12824


In [48]:
# generate mp4
fps = 45
out = 'tarragon_interp%s.mp4'%fps
with open('list.txt','w') as f:
  for i in range(count):
    f.write('file %sinterpolation/output_%05d_0.png\n'%(workbase, i))
# !ffmpeg -r $fps -f concat -safe 0 -i list.txt -c:v libx264 -pix_fmt yuv420p -profile:v baseline -movflags +faststart -r $fps $out -y
!echo ffmpeg -r $fps -f concat -safe 0 -i list.txt -c:v libx264 -pix_fmt yuv420p -profile:v baseline -movflags +faststart -r $fps $out -y

# # rename jpg
# frame = 'frame_%05d.jpg'%(sample_num-1)
# jpg = '%s.jpg'%prompt.replace(" ", "_")
# !cp $frame $jpg

ffmpeg -r 45 -f concat -safe 0 -i list.txt -c:v libx264 -pix_fmt yuv420p -profile:v baseline -movflags +faststart -r 45 tarragon_interp45.mp4 -y


In [ ]:
# # move to datestamped path
# import os, datetime
# newdir = outpath[:-1]+"_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
# !mv $outpath $newdir
# !mkdir -p $outpath

In [ ]:
# from IPython.display import display, FileLink, FileLinks

# local_file = FileLink(out.replace('"', ""), result_html_prefix="Click here to download: ")
# # local_file = FileLinks(".", result_html_suffix="?download")
# display(local_file)

In [ ]:
# from IPython.display import HTML
# local_file = out.replace('"', "")
# HTML("<a href=\""+local_file+"\">download %s</a>"%local_file)

# Notes


Based on SIREN+CLIP Colabs by: [@advadnoun](https://twitter.com/advadnoun), [@norod78](https://twitter.com/norod78)

Other CLIP notebooks: [OpenAI tutorial](https://colab.research.google.com/github/openai/clip/blob/master/Interacting_with_CLIP.ipynb), [SIREN by @advadnoun](https://colab.research.google.com/drive/1FoHdqoqKntliaQKnMoNs3yn5EALqWtvP), [SIREN by @norod78](https://colab.research.google.com/drive/1K1vfpTEvAmxW2rnhAaALRVyis8EiLOnD), [BigGAN by @advadnoun](https://colab.research.google.com/drive/1NCceX2mbiKOSlAd_o7IU7nA9UskKN5WR), [BigGAN by @eyaler](j.mp/bigclip), [BigGAN by @tg_bomze](https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/Text2Image_v2.ipynb), [BigGAN using big-sleep library by @lucidrains](https://colab.research.google.com/drive/1MEWKbm-driRNF8PrU7ogS5o3se-ePyPb), [BigGAN story hallucinator by @bonkerfield](https://colab.research.google.com/drive/1jF8pyZ7uaNYbk9ZiVdxTOajkp8kbmkLK), [StyleGAN2-ADA Anime by @nagolinc](https://colab.research.google.com/github/nagolinc/notebooks/blob/main/TADNE_and_CLIP.ipynb) [v2](https://colab.research.google.com/github/nagolinc/notebooks/blob/main/CLIP_%2B_TADNE_(pytorch)_v2.ipynb)

Using the works:

https://github.com/openai/CLIP

https://tfhub.dev/deepmind/biggan-deep-512

https://github.com/huggingface/pytorch-pretrained-BigGAN

http://www.aiartonline.com/design-2019/eyal-gruss (WanderGAN)

For a curated list of more online generative tools see: [j.mp/generativetools](https://j.mp/generativetools)


# Leftovers

Ex2. Interpolating between two vectors:

In [ ]:
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)

# num_steps = 500

# # Prepare a input
# truncation = 1.0
# workbase = '/home/jovyan/'

# # sculptures: 


# # work/visual-imagination/CLIP/class_sculpture television buddha.txt
# # work/visual-imagination/CLIP/class_a buddha sculpture with television in the grass.txt
# # work/visual-imagination/CLIP/class_television buddha sculpture with grass.txt
# # work/visual-imagination/CLIP/class_old television in grass with buddha sculpture by pacific ocean.txt
# # work/visual-imagination/CLIP/class_old television in long grass with buddha sculpture by pacific ocean.txt


# noise1 = np.loadtxt(workbase+'work/output_20210222_195143/noise_00002.txt')
# class1 = np.loadtxt(workbase+'work/output_20210222_195143/class_00002.txt')

# noise2 = np.loadtxt(workbase+'work/output_20210222_195143/noise_00008.txt')
# class2 = np.loadtxt(workbase+'work/output_20210222_195143/class_00008.txt')

# # noise_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/noise_a room with good lighting.txt')
# # class_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/class_a room with good lighting.txt')

# # interpolate
# noises = interpolate_points(noise1, noise2, num_steps)
# classes = interpolate_points(class1, class2, num_steps)

# # expand dims (only necessary for single vector)

# for i in range(num_steps):
    
#     # expand dims
#     noise_vector = np.expand_dims(noises[i], axis=0)
#     class_vector = np.expand_dims(classes[i], axis=0)

#     # convert to tensors
#     noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
#     class_vector = torch.tensor(class_vector, dtype=torch.float32)

#     print(noise_vector.shape, noise_vector.dtype, class_vector.shape, class_vector.dtype)

#     # If you have a GPU, put everything on cuda
#     noise_vector = noise_vector.to('cuda')
#     noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
#     class_vector = class_vector.to('cuda')
#     class_vector = class_vector.softmax(dim=-1)
#     model.to('cuda')

#     # Generate an image
#     with torch.no_grad():
#         output = model(noise_vector, class_vector, truncation)

#     # If you have a GPU put back on CPU
#     output = output.to('cpu')

#     # # If you have a sixtel compatible terminal you can display the images in the terminal
#     # # (see https://github.com/saitoha/libsixel for details)
#     # # display_in_terminal(output)
#     save_as_images(output, workbase+"work/output_%05d" % i)
# #     clear_output()
# #     display(JupImage("output_%05d_0.png" % i))
#     i+=1

Ex.1: Generate from a single stored noise/class vector

In [ ]:
# # random seeding
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)


# # Prepare a input
# truncation = 1.0
# workbase = '/home/jovyan/'
# noise_vector = np.loadtxt(workbase+'work/output_20210222_195143/noise_00008.txt')
# class_vector = np.loadtxt(workbase+'work/output_20210222_195143/class_00008.txt')

# # noise_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/noise_a room with good lighting.txt')
# # class_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/class_a room with good lighting.txt')



# # noise_vector = noise_vectors
# # class_vector = class_vectors

# # expand dims
# noise_vector = np.expand_dims(noise_vector, axis=0)
# class_vector = np.expand_dims(class_vector, axis=0)

# # All in tensors
# noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
# class_vector = torch.tensor(class_vector, dtype=torch.float32)

# print(noise_vector.shape, noise_vector.dtype, class_vector.shape, class_vector.dtype)

# # If you have a GPU, put everything on cuda
# noise_vector = noise_vector.to('cuda')
# noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
# class_vector = class_vector.to('cuda')
# class_vector = class_vector.softmax(dim=-1)
# model.to('cuda')

# # Generate an image
# with torch.no_grad():
#     output = model(noise_vector, class_vector, truncation)

# # If you have a GPU put back on CPU
# output = output.to('cpu')

# # # If you have a sixtel compatible terminal you can display the images in the terminal
# # # (see https://github.com/saitoha/libsixel for details)
# # # display_in_terminal(output)
# # save_as_images(output, "output_%s" % index)

# # display(Image("output_0.png"))

# # # Save results as png images
# save_as_images(output)